In [1]:
import numpy as np
import scipy.io
from scipy.io import loadmat
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, date, time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
import matplotlib.colors
import time
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from tqdm import tqdm_notebook 
from IPython.display import HTML
import warnings
from sklearn.preprocessing import OneHotEncoder
import cv2
import copy
import random
import pickle
from sklearn.datasets import make_blobs
import torch
warnings.filterwarnings('ignore')
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm 
import tarfile
import os

In [2]:
pwd

'C:\\Users\\shame\\Downloads\\MLQ5'

# a) Download Data (unpickle)

In [3]:
directory = 'C:\\Users\\shame\\Downloads\\MLQ5\\cifar-10-batches-py'

# Extracting image files to dictionary using the unpickling code given in cifar readme page:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

train_and_test_dicts = []

# f_path = os.path.join(directory, file)
# # train_and_test_dicts.append(unpickle(f_path))
# tt = unpickle(r"C:\Users\shame\Downloads\MLQ5\cifar-10-batches-py\data_batch_1")
# print(tt)

for file in os.listdir(directory):
    f_path = os.path.join(directory, file)
#     print(f_path)
    train_and_test_dicts.append(unpickle(f_path))
    
for i in range(1, len(train_and_test_dicts) + 1):
    # Creating training batches, validation batch and final test batch dictionaries:
    if i == 6:
        batch_name = "test_batch"
        print(batch_name)
        globals()[batch_name] = train_and_test_dicts[i-1]
    elif i == 5:
        batch_name = "validation_batch"
        print(batch_name)
        globals()[batch_name] = train_and_test_dicts[i-1]
    else:
        batch_name = "train_batch_" + str(i)
        print(batch_name)
        globals()[batch_name] = train_and_test_dicts[i-1]
        
# Get the metadata file:
meta_file = r'C:\Users\shame\Downloads\MLQ5\batches.meta'
# assert os.path.isfile(meta_file)
meta_data = unpickle(meta_file)

train_batch_1
train_batch_2
train_batch_3
train_batch_4
validation_batch
test_batch


In [4]:
print("Shape of data: ", train_batch_1[b'data'].shape)

Shape of data:  (10000, 3072)


# a) Normalize images

In [5]:
# Normalize the images so that the maximum and minimum pixel values are between -1 and +1.
# Referring to https://machinelearningmastery.com/how-to-manually-scale-image-pixel-data-for-deep-learning/

# To approach each set:
data = [train_batch_1, train_batch_2, train_batch_3, train_batch_4, validation_batch, test_batch]

for batch in data:
    batch[b'data'] = batch[b'data'].astype('float32')
    # NORMALIZE
    batch[b'data'] = (2.0*batch[b'data'] / 255.0) - 1
    print('Normalized data Min: %.3fu, Max: %.3f' % (batch[b'data'].min(), batch[b'data'].max()),"\n")

Normalized data Min: -1.000u, Max: 1.000 

Normalized data Min: -1.000u, Max: 1.000 

Normalized data Min: -1.000u, Max: 1.000 

Normalized data Min: -1.000u, Max: 1.000 

Normalized data Min: -1.000u, Max: 1.000 

Normalized data Min: -1.000u, Max: 1.000 



In [6]:
def get_dataset(label,data):

    data_set_test = []
    for y,x in zip(label,data):
        # Reshaping the 3072 pixels to nparrays of R, G, B pixels pf size 1024 each. 
        X_r = np.reshape(x[:1024],(32,32))
        X_g = np.reshape(x[1024:2048],(32,32))
        X_b = np.reshape(x[2048:],(32,32)) 
        # Creating 3 channels to be given as an input to the first layer of Neural Network.
        X = np.stack((X_r,X_g,X_b),0) 
        data_set_test.append([X,y])
    return data_set_test

In [7]:
#GENERATING TRAINING DATASET
appended_training_set = []
i = 0
for batch in data:
    if i<4:
        appended_training_set.append(get_dataset(batch[b'labels'], batch[b'data']))
        i += 1

training_set =[]
for i in range(len(appended_training_set)):
    for j in range(len(appended_training_set[i])):
        training_set.append(appended_training_set[i][j])       
print("Image count in training set :",len(training_set))
np.save("training_data.npy",training_set) #saving it

#GENERATING VALIDATION DATASET
validation_set = get_dataset(validation_batch[b'labels'], validation_batch[b'data'])
print("Image count in validation set :",len(validation_set))
np.save("validation_data.npy",validation_set)


#GENERATING TEST DATASET
test_set = get_dataset(test_batch[b'labels'], test_batch[b'data'])
print("Image count in test set :",len(test_set))
np.save("test_data.npy",test_set)

Image count in training set : 40000
Image count in validation set : 10000
Image count in test set : 10000


# b) Implement CNN

In [8]:
# INITIALIZING 
batch_size = 64
num_classes = 10
learning_rate = 0.02
num_epochs = 20

# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
#CREATING DATALOADERS for model

# Referred to: https://pytorch.org/docs/stable/data.html
num_workers = 5
from torch.utils.data.sampler import SubsetRandomSampler

# obtain training indices that will be used for validation
num_train = len(training_set)
train_idx = list(range(num_train))
np.random.shuffle(train_idx)

# obtain training indices that will be used for validation
num_valid = len(validation_set)
valid_idx = list(range(num_valid))
np.random.shuffle(valid_idx)

# obtain training indices that will be used for validation
num_test = len(test_set)
test_idx = list(range(num_test))
np.random.shuffle(test_idx)

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# how many samples per batch to load
batch_size = 20

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

In [10]:
#CNN MODEL

class CNN(nn.Module):
    
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        
        self.convolutional_layer_1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = 5)

        self.max_pool1 = nn.MaxPool2d(kernel_size = 2)
        
        self.convolutional_layer_2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = 5)
        
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2)
        
        self.feedforward_1 = nn.Linear(5*5*16,120)
        
        self.feedforward_2 = nn.Linear(120,84)
        
        self.feedforward_3 = nn.Linear(84,num_classes)
        
        self.softmax_final = nn.Softmax()
        
    def forward(self, x):
        
        out = self.convolutional_layer_1(x)
        
        out = self.max_pool1(F.relu(out))
        
        out = self.convolutional_layer_2(out)
        
        out = self.max_pool2(F.relu(out))
        
        # print(x.shape)
        
        out = out.view(-1, 5*5*16)
        
        out = self.feedforward_1(out)
        
        out = self.feedforward_2(F.relu(out))
        
        out = self.feedforward_3(F.relu(out))
        
        # out = self.softmax_final(out)  # Remove softmax when using CrossEntropyLoss
        
        return out        

In [11]:
# HYPERPARAMETER TUNING

#calling model
model = CNN(num_classes)
if torch.cuda.is_available():
    model.to(device)

## Set loss function:
criterion = nn.CrossEntropyLoss()

## Set optimizer:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9)

total_step = 10000

# c) TRAINING MODEL

In [12]:
valid_loss_min = np.Inf # track change in validation loss
list_train_losses = []
list_val_losses = []


for epoch in range(num_epochs):
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    class_correct_train = list(0. for i in range(10))
    class_total_train = list(0. for i in range(10))
    
    for data, target in train_loader:
        
        # move tensors to GPU if CUDA is available
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # calculate the batch loss
        loss = criterion(output, target)
        
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        
        # perform a single optimization step (parameter update)
        optimizer.step()
        
        
        # update training loss
        train_loss += loss.item()*data.size(0)
        
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)    

        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        
        # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else 
        correct = np.squeeze(correct_tensor.cpu().numpy())

        # calculate test accuracy for each object class
        for i in range(batch_size):
            label = target.data[i]
            class_correct_train[label] += correct[i].item()
            class_total_train[label] += 1        
    
#     print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    
    for data, target in valid_loader:
        
        # move tensors to GPU if CUDA is available
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        
        # calculate the batch loss
        loss = criterion(output, target)
        
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
        
        
        
        # convert output probabilities to predicted class
        _, pred = torch.max(output, 1)    

        # compare predictions to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        
        # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else 
        correct = np.squeeze(correct_tensor.cpu().numpy())

        # calculate test accuracy for each object class
        for i in range(batch_size):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1        
        
    
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    list_train_losses.append(train_loss)
    list_val_losses.append(valid_loss)

    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

    # BEST MODEL
    if valid_loss <= valid_loss_min:
        print('SAVING NEW BEST MODEL')
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss



Epoch: 0 	Training Loss: 1.779714 	Validation Loss: 1.568021
SAVING NEW BEST MODEL
Epoch: 1 	Training Loss: 1.549058 	Validation Loss: 1.598523
Epoch: 2 	Training Loss: 1.502536 	Validation Loss: 1.525787
SAVING NEW BEST MODEL
Epoch: 3 	Training Loss: 1.468298 	Validation Loss: 1.465157
SAVING NEW BEST MODEL
Epoch: 4 	Training Loss: 1.446341 	Validation Loss: 1.552579
Epoch: 5 	Training Loss: 1.439225 	Validation Loss: 1.478067
Epoch: 6 	Training Loss: 1.444138 	Validation Loss: 1.526855
Epoch: 7 	Training Loss: 1.448623 	Validation Loss: 1.562189
Epoch: 8 	Training Loss: 1.433763 	Validation Loss: 1.541322
Epoch: 9 	Training Loss: 1.460772 	Validation Loss: 1.563084
Epoch: 10 	Training Loss: 1.452895 	Validation Loss: 1.612883
Epoch: 11 	Training Loss: 1.473717 	Validation Loss: 1.636015
Epoch: 12 	Training Loss: 1.478433 	Validation Loss: 1.574545
Epoch: 13 	Training Loss: 1.486904 	Validation Loss: 1.650097
Epoch: 14 	Training Loss: 1.507689 	Validation Loss: 1.607216
Epoch: 15 	Tra

In [13]:
#TRAINING ACCURACY
print("TRAINING ACCURACY BY CLASS\n")
for i in range(10):
    if class_total_train[i] > 0:
        print('%5s: %2d%%' % (
            meta_data[b'label_names'][i], 100 * class_correct_train[i] / class_total_train[i]))
    else:
        print('Training Accuracy of %5s: N/A (no training examples)' % (meta_data[b'label_names'][i]))

print('\nOverall: %2d%%' % (
    100. * np.sum(class_correct_train) / np.sum(class_total_train)),"\n")     

#VALIDATION ACCURACY
print("VALIDATION ACCURACY BY CLASS\n")
for i in range(10):
    if class_total[i] > 0:
        print('%5s: %2d%%' % (
            meta_data[b'label_names'][i], 100 * class_correct[i] / class_total[i]))
    else:
        print('Training Accuracy of %5s: N/A (no training examples)' % (meta_data[b'label_names'][i]))

print('\nOverall: %2d%%' % (
    100. * np.sum(class_correct) / np.sum(class_total)),"\n")         
        

TRAINING ACCURACY BY CLASS

b'airplane': 45%
b'automobile': 61%
b'bird': 32%
b'cat': 32%
b'deer': 40%
b'dog': 38%
b'frog': 58%
b'horse': 53%
b'ship': 61%
b'truck': 59%

Overall: 48% 

VALIDATION ACCURACY BY CLASS

b'airplane': 59%
b'automobile': 54%
b'bird': 20%
b'cat': 12%
b'deer': 57%
b'dog': 34%
b'frog': 48%
b'horse': 52%
b'ship':  7%
b'truck': 56%

Overall: 40% 



In [14]:
model.load_state_dict(torch.load('model_cifar.pt'))

<All keys matched successfully>

In [15]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

for data, target in test_loader:
    
    # move tensors to GPU if CUDA is available
    if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
    
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    
    # calculate the batch loss
    loss = criterion(output, target)
    
    # update average validation loss 
    test_loss += loss.item()*data.size(0)
    
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    
    # correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else 
    correct = np.squeeze(correct_tensor.cpu().numpy())
    
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

#         print('Loss: {:.4f}'.format(loss.item()))

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

#VALIDATION ACCURACY
print("TESTING ACCURACY BY CLASS\n")
for i in range(10):
    if class_total[i] > 0:
        print('%5s: %2d%%' % (
            meta_data[b'label_names'][i], 100 * class_correct[i] / class_total[i]))
    else:
        print('Training Accuracy of %5s: N/A (no training examples)' % (meta_data[b'label_names'][i]))

print('\nOverall: %2d%%' % (
    100. * np.sum(class_correct) / np.sum(class_total)),"\n")  

Test Loss: 1.460338

TESTING ACCURACY BY CLASS

b'airplane': 58%
b'automobile': 73%
b'bird': 31%
b'cat': 32%
b'deer': 25%
b'dog': 43%
b'frog': 62%
b'horse': 52%
b'ship': 50%
b'truck': 50%

Overall: 48% 

